In [1]:
import sys; sys.path.append("../")

import pandas as pd

from lib.name import clean_name
from lib.excel import remove_unnamed_cols
from lib.date import to_datetime_series
from lib.match_records import gen_uid
from lib.columns import (
    rearrange_personel_columns, rearrange_personel_history_columns
)

In [2]:
df1 = pd.read_excel("../data/New Orleans Civil Service Department/Administrative Data/New Orleans_CSD_PPRR_2009.xlsx")
df1 = remove_unnamed_cols(df1)
df1.rename(columns={name: name.title() for name in df1.columns}, inplace=True)
df1

,Orgn Desc,Last Name,First Name,Titl E Code,Title Desc,Term Date,Pay Prog Start Date,Annual Salary
0,FISCAL MANAGEMENT,ABBOTT,CHRISTOPHER,C7134,POLICE OFFICER 4,NaT,1992-09-02,46885.0056
1,TRAFFIC,ABBOTT,DAVID,C7134,POLICE OFFICER 4,NaT,1987-01-11,48064.4565
2,POLICE RECRUITS,ABRAM,SHELTON,C7109,POLICE RECRUIT ACADEMY,NaT,2009-06-09,34797.9137
3,FISCAL MANAGEMENT,ADAMS,JAMES,C7133,POLICE OFFICER 3,2009-08-30,1987-06-09,45734.4971
4,PUBLIC INTEGRITY,ADAMS,BRUCE,U0018,ASSISTANT SUPT OF POLICE,NaT,1975-10-16,106440.2966
...,...,...,...,...,...,...,...,...
1976,POLICE RECRUITS,YOUNT,JEFFREY,NaN,POLICE RECRUIT ACADEMY,NaT,NaT,34797.9137
1977,TECHNICIAL SERVICES BUREA,ZALDIVAR,SARA,NaN,POLICE COMPLAINT OPERATOR,NaT,NaT,25508.0588
1978,SIXTH DISTRICT,ZAPICO,DANIEL,NaN,POLICE OFFICER 1,NaT,NaT,36570.7051
1979,THIRD DISTRICT,ZOLLER,RON,NaN,POLICE OFFICER 4,NaT,NaT,44612.2209


In [3]:
df2 = df1[["Orgn Desc", "First Name", "Last Name", "Titl E Code", "Title Desc", "Term Date", "Pay Prog Start Date", "Annual Salary"]]
df2.rename(columns={
    "Term Date": "Termination Date",
    "Orgn Desc": "Department",
    "Titl E Code": "Rank Number #",
    "Title Desc": "Rank",
}, inplace=True)
for col in ["Last Name", "First Name"]:
    df2.loc[:, col] = df2[col].map(clean_name, na_action="ignore")
for col in ["Termination Date", "Pay Prog Start Date"]:
    df2.loc[:, col] = to_datetime_series(df2[col])
df2 = gen_uid(df2, ["Last Name", "First Name"])
personel = rearrange_personel_columns(df2)
history = rearrange_personel_history_columns(df2)

In [4]:
personel

,UID,Last Name,First Name
0,f596663dc3,Abbott,Christopher
1,dfb1a08325,Abbott,David
2,b8566ab933,Abram,Shelton
3,75953b7ab7,Adams,James
4,54cf218c0b,Adams,Bruce
...,...,...,...
1506,ed1dbff904,Yount,Jeffrey
1507,4b6f2f301a,Zaldivar,Sara
1508,0f2b01d7a4,Zapico,Daniel
1509,1ebbd049e0,Zoller,Ron


In [5]:
history

,UID,Department,Rank Number #,Rank,Termination Date,Pay Prog Start Date,Annual Salary
0,f596663dc3,FISCAL MANAGEMENT,C7134,POLICE OFFICER 4,NaT,1992-09-02,46885.0056
1,dfb1a08325,TRAFFIC,C7134,POLICE OFFICER 4,NaT,1987-01-11,48064.4565
2,b8566ab933,POLICE RECRUITS,C7109,POLICE RECRUIT ACADEMY,NaT,2009-06-09,34797.9137
3,75953b7ab7,FISCAL MANAGEMENT,C7133,POLICE OFFICER 3,2009-08-30,1987-06-09,45734.4971
4,54cf218c0b,PUBLIC INTEGRITY,U0018,ASSISTANT SUPT OF POLICE,NaT,1975-10-16,106440.2966
...,...,...,...,...,...,...,...
1972,ed1dbff904,POLICE RECRUITS,NaN,POLICE RECRUIT ACADEMY,NaT,NaT,34797.9137
1973,4b6f2f301a,TECHNICIAL SERVICES BUREA,NaN,POLICE COMPLAINT OPERATOR,NaT,NaT,25508.0588
1974,0f2b01d7a4,SIXTH DISTRICT,NaN,POLICE OFFICER 1,NaT,NaT,36570.7051
1975,1ebbd049e0,THIRD DISTRICT,NaN,POLICE OFFICER 4,NaT,NaT,44612.2209
